In [1]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import numpy as np
import math

In [2]:
##Importo para df costos Andesmar
#df troncal
dfc_tron_and = pd.read_excel('cos_tronc_and.xlsx')
#df entrega
dfc_ent_and = pd.read_excel('cos_ent_and.xlsx')
#df nomenclador
df_nom_and = pd.read_excel ('nom_and.xlsx')

In [3]:
##Completo Codigos Postales que no se encuentran en nomeclador

df_nom_and = df_nom_and.append({'Codigo Postal': 2013,
             'Provincia': 'SANTA FE',
             'TRONCAL':'Zona I',
             'ULTIMA MILLA':'Cabecera',
             'Operador':'LH-Paquery'}, ignore_index=True)

df_nom_and = df_nom_and.append({'Codigo Postal': 2189,
             'Provincia': 'CORDOBA',
             'TRONCAL':'Zona II',
             'ULTIMA MILLA':'Interior II',
             'Operador':'Andesmar'}, ignore_index=True)

df_nom_and = df_nom_and.append({'Codigo Postal': 5019,
             'Provincia': 'CORDOBA',
             'TRONCAL':'Zona II',
             'ULTIMA MILLA':'Cabecera',
             'Operador':'Andesmar'}, ignore_index=True)

In [4]:
dfc_tron_and = pd.melt(dfc_tron_and, id_vars=['Tamaño_and',
                                                'Tamaño_and_2',
                                                'peso_and',
                                                'zona_and',
                                                'año_and'],
        var_name='mes_and',
        value_name='costo_tron')

In [5]:
dfc_ent_and = pd.melt(dfc_ent_and, id_vars=['Tamaño_and',
                                                'Tamaño_and_2',
                                                'peso_and',
                                                'año_and',
                                           'sub-zona_and'],
        var_name='mes_and',
        value_name='costo_ent')

In [6]:
dfc_and = pd.merge(df_nom_and, dfc_tron_and, how='left', left_on='TRONCAL', right_on='zona_and', sort=True)

In [7]:
#cols_dfc_and = dfc_ent_and.columns.difference(dfc_and.columns)
#dfNew = merge(dfc_and, dfc_ent_and[cols_dfc_and], left_index=True, right_index=True, how='outer')


dfc_and = pd.merge(dfc_and, dfc_ent_and, how='left', left_on=['ULTIMA MILLA', 'Tamaño_and', 'Tamaño_and_2', 'peso_and','mes_and', 'año_and'],
                   right_on=['sub-zona_and', 'Tamaño_and', 'Tamaño_and_2', 'peso_and','mes_and', 'año_and'], sort=True)

In [8]:
dfpx = pd.read_excel ('px 051120.xlsx') ##CAMBIO MATRIZ COMPLETA PARA TODA LA HISTORIA

In [9]:
dfpx['Año'] = 2020

In [10]:
dfpx.rename(columns={'Febrero':2, 'Marzo':3, 'Abril':4,
       'Mayo':5, 'Junio':6, 'Julio':7, 'Agosto':8, 'Septiembre':9, 'Octubre':10,
       'Noviembre':11, 'Diciembre':12}, inplace=True)

In [11]:
##Cargo enero igual a febrero
dfpx[1] = dfpx[2]

In [12]:
dfpx.drop(['Macheo Facturación (Archivo Matriz)'], axis=1, inplace=True)
dfpx.drop(['Propietario'], axis=1, inplace=True)
dfpx.drop(['CUIT'], axis=1, inplace=True)

##Parche que usa Bonzo para cerrar el mes con los precios de plataforma
dfpx = dfpx.loc[~(dfpx['Razón Social']=='Para macheo Agosto')]
dfpx.drop(['Razón Social'], axis=1, inplace=True)

In [13]:
dfpx['Ámbito Geográfico'] = dfpx['Ámbito Geográfico'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfpx['Cliente'] = dfpx['Cliente'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [14]:
##Empato los tipos de servicios de la app con los de dataset precios
dfts = pd.read_excel ('Tipo de servicio.xlsx')

dfpx['Tipo de Servicio'] = dfpx['Tipo de Servicio'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfts['Tipo de Servicio'] = dfts['Tipo de Servicio'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfts['Tipo_app_px'] = dfts['Tipo_app_px'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

dfpx = pd.merge(dfpx, dfts, how='left', left_on='Tipo de Servicio', right_on='Tipo de Servicio', sort=True)

In [15]:
dfts

,Tipo_app_px,Tipo de Servicio
0,enviar,paq_delivery
1,store_withdrawal,paq_store
2,pick_up,paq_pick_up
3,enviar,uso_de_plataforma
4,flex,flex


In [16]:
#df precios pick up
dfpick = dfpx.loc[(dfpx['Tipo de Servicio']=='paq_pick_up')]

In [17]:
dfpx = dfpx.append({'Razón Social': 'BE ENTERPRISES S.A.',
             'CUIT': '30-71007968-0',
             'Cliente':'Akiabara',
             'Tipo de Servicio':'paq_delivery',
             'Plazo':'Mismo dia',
             'Ámbito Geográfico': 'CABA',
             'Tamaño': 'Chico',
             'Año': 2020, 8: 130}, ignore_index=True)
dfpx = dfpx.append({'Razón Social': 'BE ENTERPRISES S.A.',
             'CUIT': '30-71007968-0',
             'Cliente':'Akiabara',
             'Tipo de Servicio':'paq_delivery',
             'Plazo':'Mismo dia',
             'Ámbito Geográfico': 'AMBA I',
             'Tamaño': 'Chico',
             'Año': 2020, 8: 169}, ignore_index=True)
dfpx = dfpx.append({'Razón Social': 'BE ENTERPRISES S.A.',
             'CUIT': '30-71007968-0',
             'Cliente':'Akiabara',
             'Tipo de Servicio':'paq_delivery',
             'Plazo':'Mismo dia',
             'Ámbito Geográfico': 'AMBA II',
             'Tamaño': 'Chico',
             'Año': 2020, 8: 219}, ignore_index=True)
dfpx = dfpx.append({'Razón Social': 'BE ENTERPRISES S.A.',
             'CUIT': '30-71007968-0',
             'Cliente':'Akiabara',
             'Tipo de Servicio':'paq_store',
             'Plazo':'Plazo 48 hs',
             'Ámbito Geográfico': 'CABA',
             'Tamaño': 'Chico',
             'Año': 2020, 8: 206*.6}, ignore_index=True)
dfpx = dfpx.append({'Razón Social': 'BE ENTERPRISES S.A.',
             'CUIT': '30-71007968-0',
             'Cliente':'Akiabara',
             'Tipo de Servicio':'paq_store',
             'Plazo':'Plazo 48 hs',
             'Ámbito Geográfico': 'AMBA I',
             'Tamaño': 'Chico',
             'Año': 2020, 8: 379*.6}, ignore_index=True)
dfpx = dfpx.append({'Razón Social': 'BE ENTERPRISES S.A.',
             'CUIT': '30-71007968-0',
             'Cliente':'Akiabara',
             'Tipo de Servicio':'paq_store',
             'Plazo':'Plazo 48 hs',
             'Ámbito Geográfico': 'AMBA I',
             'Tamaño': 'Grande',
             'Año': 2020, 8: (379*1.3*1.3)*.6}, ignore_index=True)

In [18]:
dfpx = dfpx.append({'Razón Social': 'ANDESMAR CARGAS SOCIEDAD ANONIMA',
             'CUIT': '30-60150277-8',
             'Cliente':'Andesmar',
             'Tipo de Servicio':'paq_delivery',
             'Plazo':'Plazo 48 hs',
             'Ámbito Geográfico': 'Mendoza Cabecera',
             'Tamaño': 'Chico',
             'Año': 2020, 8: 78}, ignore_index=True)
dfpx = dfpx.append({'Razón Social': 'ANDESMAR CARGAS SOCIEDAD ANONIMA',
             'CUIT': '30-60150277-8',
             'Cliente':'Andesmar',
             'Tipo de Servicio':'paq_delivery',
             'Plazo':'Plazo 48 hs',
             'Ámbito Geográfico': 'Mendoza Interior',
             'Tamaño': 'Chico',
             'Año': 2020, 8: 78}, ignore_index=True)

In [19]:
##Elimino los tipos de servicios que no tienen utilidad para los paquetes

dfpx = dfpx.loc[~(dfpx['Tipo de Servicio']=='paq_pick_up') &
                ~(dfpx['Tipo de Servicio']=='paq_collect') &
                ~(dfpx['Tipo de Servicio']=='paq_point') &
                ~(dfpx['Tipo de Servicio']=='paq_retiro') &
                ~(dfpx['Tipo de Servicio']=='seguro') &
                ~(dfpx['Tipo de Servicio']=='entrega_a_locales') &
                ~(dfpx['Tipo de Servicio']=='uso_de_plataforma')]

In [20]:
dfpx = pd.melt(dfpx, id_vars=['Razón Social',
                                      'CUIT',
                                   'Cliente',
                          'Tipo de Servicio',
                                     'Plazo',
                         'Ámbito Geográfico',
                                    'Tamaño',
                                       'Año',
            'Rango de paquetes (Plataforma)',
                              'Tipo_app_px'],
        var_name='Mes',
        value_name='Precio')

In [21]:
dfpx.dropna(subset=['Precio'], inplace=True)
dfpx['Linea_px'] = 'Paquetes'

In [22]:
dfpx['Tipo_app_px'].head().value_counts()

flex    5
Name: Tipo_app_px, dtype: int64

In [23]:
#dfpx_pl['Rango_paquetes'] = dfpx_pl['Rango de paquetes (Plataforma)'].str.extract('(\d+)')

In [24]:
inicio = datetime.now()
dfpq1 = pd.read_excel('ene20 a jul20.xlsx')
dfpq2 = pd.read_excel('ago20.xlsx')
dfpq3 = pd.read_excel('sep20.xlsx')
dfpq4 = pd.read_excel('oct20.xlsx')
datetime.now() - inicio

datetime.timedelta(seconds=133, microseconds=783872)

In [25]:
frames = [dfpq1, dfpq2, dfpq3, dfpq4]

dfpq = pd.concat(frames, sort=True, ignore_index=True)

In [26]:
dfpq.shape

(337184, 33)

In [27]:
dfpq['Empresa'] = dfpq['Empresa'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfpq['OPL Entrega'] = dfpq['OPL Entrega'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfpq['OPL Paquer'] = dfpq['OPL Paquer'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfpq['Tipo'] = dfpq['Tipo'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [28]:
#Existe el Recibir?
dfpq.loc[(dfpq['Tipo']=='recibir'), 'Tipo'] = 'enviar'

dfpq.loc[((dfpq['Empresa']=='ups') & (dfpq['Tipo']=='store_withdrawal')), 'Tipo'] = 'enviar'
dfpq.loc[((dfpq['Empresa']=='dvigi') & (dfpq['Tipo']=='store_withdrawal')), 'Tipo'] = 'enviar'

In [29]:
dfpq['Creación']= pd.to_datetime(dfpq['Creación'])
dfpq['Fecha de Ingreso a Paquery']= pd.to_datetime(dfpq['Fecha de Ingreso a Paquery']) 
dfpq['Arribo a Paquery Point']= pd.to_datetime(dfpq['Arribo a Paquery Point'])
dfpq['Fecha en Poder']= pd.to_datetime(dfpq['Fecha en Poder'])
dfpq['Fecha de entrega']= pd.to_datetime(dfpq['Fecha de entrega']) 
dfpq['2da Visita']= pd.to_datetime(dfpq['2da Visita'])
dfpq['1er Visita']= pd.to_datetime(dfpq['1er Visita'])

In [30]:
dfpq_fl = pd.read_excel('fl_oct20.xlsx')

In [31]:
dfpq_fl['Tipo'] = dfpq_fl['Tipo'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [32]:
#Creo un código único para el mes
dfpq_fl['Código Externo'] = dfpq_fl['Código Externo'] + dfpq_fl['Fecha de entrega']

In [33]:
frames = [dfpq, dfpq_fl]

dfpq = pd.concat(frames, sort=True, ignore_index=True)

In [34]:
dfpq.shape

(343006, 33)

In [35]:
dfpq['Código Externo'] = dfpq['Código Externo'].astype(str)

In [36]:
##Hay paquetes "inversa" mal cargados que trae "PAQUERY" como Empresa
dfpq['Empresa'] = np.where(dfpq['Empresa'] == 'paquery', dfpq['Nombre destinatario'], dfpq['Empresa'])

In [37]:
##Hay paquetes que tienen más de un paquete y en el código externo incluye la palabra 3 PAQUETES, 7 PAQUETES, etc. Hay que pasarlos a tamaño grande
dfpq['Tamaño'] = np.where(dfpq['Código Externo'].str.contains('PAQUETES'), 'Grande', dfpq['Tamaño'])

In [38]:
##Cuando se juntan df de distintos meses y hay paquetes creados el mes anterior pero entregados o con 2da visita
##el mes siguiente se van a duplicar. Puede haber cancelados con inversa que dupliquen y hay que verlos.

#Primero elimino los duplicados, cancelados que no tienen 2da visita. Los cancelados que no están duplicados (aunque no tengan)
#segunda visita, si son transacción, los tengo que ver.
dfpq = dfpq.loc[~((dfpq['Código Externo'].duplicated()) & (dfpq['Estado']=='Cancelado') & (dfpq['2da Visita'].isnull()))]

#Depués me quedo con el más nuevo
dfpq.drop_duplicates(subset=['Código Externo', 'Estado'], keep='last', inplace=True) 

##Si tengo un cancelado y un devuelto con 2da visita, pago uno sólo de esos.
dfpq.drop_duplicates(subset=['Código Externo', '2da Visita'], keep='last', inplace=True)

In [39]:
##Dataset de clientes mayoristas
dfcl3 = pd.read_excel ('Clientes 3ero.xlsx')

In [40]:
dfcl3['Clientes 3ero'] = dfcl3['Clientes 3ero'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfcl3['Cliente_mayorista'] = dfcl3['Cliente_mayorista'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [41]:
dfpq['Empresa'] = dfpq['Empresa'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [42]:
dfpqcl3 = pd.merge(dfpq, dfcl3, how='left', left_on='Empresa', right_on='Clientes 3ero', sort=True)

In [43]:
##Si son clientes de un cliente mayorista, se le factura al mayorista
dfpqcl3['Cliente_facturacion'] = np.where(dfpqcl3['Clientes 3ero'].isnull(), dfpqcl3['Empresa'], dfpqcl3['Cliente_mayorista'])

In [44]:
dfpqcl3.loc[dfpqcl3.Empresa == 'la_sante_(san_martin)', 'Cliente_facturacion'] = 'farmacia_la_sante'
dfpqcl3.loc[dfpqcl3.Empresa == 'la_sante_(devoto/colegiales)', 'Cliente_facturacion'] = 'farmacia_la_sante'

In [45]:
new = dfpqcl3['Zona'].str.split(" - ", expand=True)
dfpqcl3['Zona_ppal'] = new[0]
dfpqcl3['Zona_sec'] = new[1]

In [46]:
dfpqcl3.loc[(dfpqcl3.Zona_ppal == 'Rosario') & (dfpqcl3.Zona_sec == 'Capital'), 'Zona_ppal'] = 'Rosario Capital'

In [47]:
dfpqcl3.columns

Index(['1er Visita', '2da Visita', 'Arribo a Paquery Point', 'Cant. de Bultos',
       'Codigo Postal', 'Costo Estimado', 'Creación', 'Código Externo',
       'Descripción', 'Destino Comentarios', 'Destino Dir Comentarios',
       'Dirección Origen', 'Dirección destino', 'Empresa', 'Estado',
       'Fecha de Ingreso a Paquery', 'Fecha de entrega', 'Fecha en Poder',
       'Motivo 1er visita', 'Motivo 2da visita', 'Nombre Origen',
       'Nombre Paquer', 'Nombre destinatario', 'OPL Entrega', 'OPL Paquer',
       'Paquer', 'Paquery Point', 'Plazo', 'Rollcontainer/Posición', 'Tamaño',
       'Tipo', 'Usuario', 'Zona', 'Clientes 3ero', 'Cliente_mayorista',
       'Cliente_facturacion', 'Zona_ppal', 'Zona_sec'],
      dtype='object')

In [48]:
dfpqcl3['Zona'] = dfpqcl3['Zona'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [49]:
##Dataset de Zonas vs Ámbito geográfico
#dfz = pd.read_excel ('Zonas_completo.xlsx', sheet_name='Zona final')
dfz = pd.read_excel ('cod_postal_x_zona.xlsx')

In [50]:
dfz['Zona_cod'] = dfz['Zona_cod'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfz['ambito_geo_facturacion'] = dfz['ambito_geo_facturacion'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfz['ambito_geo_costos'] = dfz['ambito_geo_costos'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [51]:
##Genero df para el Codigo Postal por default, tomando la zona de entrega
cod_default = dfpqcl3['Codigo Postal'].groupby([dfpqcl3.Zona]).agg('min').to_frame(name = 'cod_default').astype(int).reset_index()
cod_default.rename(columns={'Zona':'Zona_cod_def'}, inplace=True)

In [52]:
#Mergeo paquetes con el default de Codigo Postal
dfpqcl3 = pd.merge(dfpqcl3, cod_default, how='left', left_on='Zona', right_on='Zona_cod_def', sort=True)

In [53]:
##Completo los NA en Codigo Postal por el Codigo Default
dfpqcl3['Codigo Postal'].fillna(dfpqcl3['cod_default'], inplace=True)

##Completo los NA de Codigo Postal cuando tampoco tengo Zona, es decir no tengo Codigo Default
dfpqcl3['Codigo Postal'].fillna(1000, inplace=True)
dfpqcl3['Zona'].fillna('caba_sur', inplace=True)

In [54]:
dfpqz = pd.merge(dfpqcl3, dfz, how='left', left_on='Codigo Postal', right_on='cod_postal', sort=True)

In [55]:
dfpqp = pd.read_excel ('Zonas_pqp.xlsx')

In [56]:
dfpqp['Zona_pqp'] = dfpqp['Zona_pqp'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfpqp['OPL_default_pqp'] = dfpqp['OPL_default_pqp'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [57]:
dfpqz['Paquery Point'].unique()

array(['PaQuery Point - Zona Sur', 'PaQuery Point - CABA', nan,
       'PaQuery Point - Zona Norte', 'PaQuery Point - Munro',
       'PaQuery Point - Mercado Central', 'PaQuery Point - Mendoza',
       'PaQuery Point - Encopress', 'PaQuery Point - Lion',
       'PaQuery Point - Escobar', 'PaQuery Point InterExpress - La Plata',
       'PaQuery Point - Andesmar', 'PaQuery Point - MDQ',
       'PaQuery Point - Rosario', 'PaQuery Point - Cordoba',
       'Paquery Point - Mendoza TARJETA SUBE',
       'PaQuery Point - Mar del Plata'], dtype=object)

In [58]:
dfpqp['Paquery_Point'].unique()

array(['PaQuery Point - Andesmar', 'PaQuery Point - CABA',
       'PaQuery Point - Encopress', 'PaQuery Point - Escobar',
       'PaQuery Point - Lion', 'PaQuery Point - MDQ',
       'PaQuery Point - Mendoza', 'PaQuery Point - Mercado Central',
       'PaQuery Point - Munro', 'PaQuery Point - Rosario',
       'PaQuery Point - Zona Norte', 'PaQuery Point - Zona Sur',
       'PaQuery Point InterExpress - La Plata', 'PaQuery Point - Cordoba',
       'Paquery Point - Mendoza TARJETA SUBE',
       'PaQuery Point - Mar del Plata'], dtype=object)

In [59]:
dfpqz = pd.merge(dfpqz, dfpqp, how='left', left_on='Paquery Point', right_on='Paquery_Point', sort=True)

In [60]:
##Default de Zona del Paquery Point igual al ámbito costos, es decir Zona I 
dfpqz['Zona_pqp'].fillna(dfpqz['ambito_geo_costos'], inplace=True)

##Regla de Zona para costos OPL
dfpqz.loc[dfpqz['Zona_pqp'] == dfpqz['ambito_geo_costos'], 'Zona_clasif_OPL'] = 'Zona I'
dfpqz.loc[dfpqz['Zona_pqp'] != dfpqz['ambito_geo_costos'], 'Zona_clasif_OPL'] = 'Zona II'

In [61]:
##Creo df de dafault para el ambito de facturación
##A partir del df de codigos postales
dfz_default = dfz.drop(['cod_postal', 'ambito_geo_costos'], axis=1)

##Agrupo
dfz_default = dfz_default.groupby('Zona_cod').first().reset_index()

##Cambio nombre de campos para no duplicar
dfz_default.rename(columns={'Zona_cod': 'Zona_cod_def_ambito', 'ambito_geo_facturacion': 'ambito_geo_fact_def'}, inplace=True)

In [62]:
dfz_default['Zona_cod_def_ambito'].unique

<bound method Series.unique of 0                   bahia_blanca
1                     caba_norte
2                     caba_oeste
3                       caba_sur
4                          chaco
                 ...            
58                      san_luis
59    san_nicolas_de_los_arroyos
60            santa_fe_-_capital
61          santa_fe_-_esperanza
62                       tucuman
Name: Zona_cod_def_ambito, Length: 63, dtype: object>

In [63]:
#Mergeo paquetes con el default de Ámbito geo facturación
dfpqz = pd.merge(dfpqz, dfz_default, how='left', left_on='Zona', right_on='Zona_cod_def_ambito', sort=True)

In [64]:
##Completo los NA de ambito geo fact con el default creado
dfpqz['ambito_geo_facturacion'].fillna(dfpqz['ambito_geo_fact_def'], inplace=True)

##Completo NA luego de default con CABA
dfpqz['ambito_geo_facturacion'].fillna('CABA', inplace=True)

In [65]:
##Clientes que un OPL carga en la plataforma y opera por su cuenta
#dfcl = pd.read_excel ('Clientes de OPLs.xlsx')

##Para distinguir el uso de plataforma
dfprop = pd.read_excel ('Propietario_cliente.xlsx')

In [66]:
dfprop['Cliente_propietario'] = dfprop['Cliente_propietario'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfprop['Propietario'] = dfprop['Propietario'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [67]:
dfpqz['Cliente_facturacion'] = dfpqz['Cliente_facturacion'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfpqz['OPL Entrega'] = dfpqz['OPL Entrega'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfpqz['OPL Paquer'] = dfpqz['OPL Paquer'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [68]:
dfpqprop = pd.merge(dfpqz, dfprop, how='left', left_on='Cliente_facturacion', 
                    right_on='Cliente_propietario', sort=True) ##Cambie empresa por cliente_fact !!!

In [69]:
#Creo default de OPL Entrega como el Propietario
dfpqprop.loc[dfpqprop['OPL Entrega'].isnull(), 'OPL Entrega'] = dfpqprop['Propietario'] ################################

In [70]:
##Creo OPL pago para machear precio de la historia previa a la inclusion OPL Entrega
##y los null
dfpqprop.loc[(dfpqprop['OPL Entrega'] == 'paquery') |
         (dfpqprop['OPL Paquer'] == 'paquery') & (dfpqprop['OPL Entrega'].isnull()) |
         (dfpqprop['OPL Paquer'].isnull()) & (dfpqprop['OPL Entrega'].isnull()), 'OPL_pago'] = 'paquery'

dfpqprop.loc[(dfpqprop['OPL Entrega'] == 'star-division'), 'OPL_pago'] = 'star-division'

dfpqprop['OPL_pago'].fillna('otro', inplace=True)

In [71]:
print(dfpqprop['Código Externo'].count() - len(dfpqprop['Código Externo'].unique()))
print(dfpqprop.loc[dfpqprop['Código Externo'] == 'FF185248832048'].shape[0])

20
1


In [72]:
##Cargo mes y año de la segunda visita en mes y año de entrega para que me tome el precio
##cuando cruzo los df de paquetes y precios.
#Necesito que tenga fecha para que me de precio y poder cobrar los paquetes no entregados pero con dos visitas
dfpqprop.loc[~(dfpqprop['Fecha de entrega'].isnull()), 'fecha_fc'] = dfpqprop['Fecha de entrega']

dfpqprop.loc[((dfpqprop['Fecha de entrega'].isnull()) & 
              ~(dfpqprop['2da Visita'].isnull())), 'fecha_fc'] = dfpqprop['2da Visita']

#Cuando ambos son nulos (pueden ser devueltos, algun otro?)
#En poder
dfpqprop.loc[((dfpqprop['Fecha de entrega'].isnull()) & 
              (dfpqprop['2da Visita'].isnull())) &
             ~(dfpqprop['Fecha en Poder'].isnull()), 'fecha_fc'] = dfpqprop['Fecha en Poder']

#Arribo
dfpqprop.loc[((dfpqprop['Fecha de entrega'].isnull()) & 
              (dfpqprop['2da Visita'].isnull())) &
              (dfpqprop['Fecha en Poder'].isnull()) &
             ~(dfpqprop['Arribo a Paquery Point'].isnull()), 'fecha_fc'] = dfpqprop['Arribo a Paquery Point']

#Ingreso
dfpqprop.loc[((dfpqprop['Fecha de entrega'].isnull()) & 
              (dfpqprop['2da Visita'].isnull())) &
              (dfpqprop['Fecha en Poder'].isnull()) &
              (dfpqprop['Arribo a Paquery Point'].isnull()) &
             ~(dfpqprop['Fecha de Ingreso a Paquery'].isnull()), 'fecha_fc'] = dfpqprop['Fecha de Ingreso a Paquery']

#Creacion
dfpqprop.loc[(dfpqprop['Fecha de entrega'].isnull()), 'fecha_fc'] = dfpqprop['Creación']

In [73]:
#Cargo mes y año de entrega para cruzar con matriz de precios
dfpqprop['año_fc'] = pd.DatetimeIndex(dfpqprop['fecha_fc']).year
dfpqprop['mes_fc'] = pd.DatetimeIndex(dfpqprop['fecha_fc']).month
dfpqprop['dia_fc'] = pd.DatetimeIndex(dfpqprop['fecha_fc']).day

In [74]:
dfpqprop['mes_fc'].isnull().value_counts()

False    337022
Name: mes_fc, dtype: int64

In [75]:
dfpqprop['OPL Entrega'].isna().value_counts()

False    336073
True        949
Name: OPL Entrega, dtype: int64

In [76]:
##Clientes de Star-division que no tienen OPL Entrega son por default de Star-division (Transacciones)
#dfpqprop.loc[(dfpqprop['Propietario'] == 'star-division') & (dfpqprop['OPL Entrega'].isna()), 'OPL Entrega'] = 'star-division'

In [77]:
dfpqprop.loc[(dfpqprop['Propietario'] == 'star-division')]['OPL Entrega'].isna().value_counts()

False    14230
Name: OPL Entrega, dtype: int64

In [78]:
dfpqprop['Linea'] = dfpqprop['OPL Entrega'] == dfpqprop['Propietario']
dfpqprop.loc[dfpqprop.Linea == True, 'Linea'] = 'Transacción'
dfpqprop.loc[dfpqprop.Linea == False, 'Linea'] = 'Paquetes'

In [79]:
dfpqprop.loc[(dfpqprop.Propietario == 'paquery') & 
             (dfpqprop['OPL Entrega'] == 'paquery'), 'Linea'] = 'Paquetes'

dfpqprop.loc[(dfpqprop.Tipo == 'flex'), 'Linea'] = 'Paquetes'

#Los paquetes de interior de Andesmar tienen OPL Entrega Paquery pero son transacciones
dfpqprop.loc[(dfpqprop.Propietario == 'andesmar') & 
             ((dfpqprop['Empresa'] == 'andesmar') | dfpqprop['OPL Entrega'].isnull()), 'Linea'] = 'Transacción'

#Paquetes de Quilmes (Siempre en casa) que tienen empresa Siempre en casa y OPL Paquery, en realidad son de Andesmar
dfpqprop.loc[(dfpqprop.Propietario == 'andesmar') & 
             (dfpqprop['Empresa'] == 'siempre_en_casa'), 'Linea'] = 'Transacción'

dfpqprop.loc[(dfpqprop.Propietario == 'andesmar') & 
             (dfpqprop['Empresa'] == 'quilmes'), 'Linea'] = 'Transacción'

#Paquetes de Sube que tienen empresa Sube y OPL Paquery, en realidad son de Andesmar
dfpqprop.loc[(dfpqprop.Propietario == 'andesmar') & 
             (dfpqprop['Empresa'] == 'sube'), 'Linea'] = 'Transacción'

dfpqprop.loc[(dfpqprop.Propietario == 'andesmar') & 
             (dfpqprop['Empresa'] == 'diario_los_andes'), 'Linea'] = 'Transacción'

In [80]:
#Antes de octubre no se cobraban ni las transacciones canceladas ni los paquetes
dfpqprop = dfpqprop.loc[~((dfpqprop['Estado']=='Cancelado') & dfpqprop['2da Visita'].isnull()  
                        & (dfpqprop['mes_fc'] < 10))]

#Después de octubre las transacciones canceladas se empezaron a cobrar, los paquetes no
dfpqprop = dfpqprop.loc[~((dfpqprop['Estado']=='Cancelado') & dfpqprop['2da Visita'].isnull() 
                        & (dfpqprop['Linea']=='Paquetes') & (dfpqprop['mes_fc'] >= 10))]

#~((dfpqprop['Estado']=='Devuelto') & dfpqprop['2da Visita'].isnull()) & #############HAY QUE COBRARLOS

In [81]:
dfpqprop[['Estado', 'Creación', 'Fecha de Ingreso a Paquery', 
          'Arribo a Paquery Point', 'Fecha en Poder']].loc[dfpqprop['mes_fc'].isnull()]

,Estado,Creación,Fecha de Ingreso a Paquery,Arribo a Paquery Point,Fecha en Poder


In [82]:
dfpqprop.loc[(dfpqprop['mes_fc'].isnull() & dfpqprop['Fecha en Poder'].isnull())].shape[0]

0

In [83]:
dfpqprop['OPL Entrega'].isnull().value_counts()

False    325199
True        716
Name: OPL Entrega, dtype: int64

In [84]:
dfpqprop['Linea'].value_counts()

Paquetes       266722
Transacción     59193
Name: Linea, dtype: int64

In [85]:
##Calculo la cantidad de paquetes para el uso de plataforma
tblxprop = dfpqprop['Linea'].loc[dfpqprop.Linea == 'Transacción'].groupby([dfpqprop.año_fc, dfpqprop.mes_fc, 
                                               dfpqprop.Propietario]).agg('count').to_frame(name = 'Pq x prop').reset_index()

In [86]:
tblxprop

,año_fc,mes_fc,Propietario,Pq x prop
0,2020,4,andesmar,416
1,2020,5,andesmar,1355
2,2020,5,encopress,148
3,2020,5,star-division,510
4,2020,6,andesmar,3458
5,2020,6,encopress,634
6,2020,6,star-division,1283
7,2020,7,andesmar,7202
8,2020,7,encopress,6
9,2020,7,star-division,1784


In [87]:
tblxprop['Linea_pl'] = 'Transacción'

In [88]:

dfpqprop = pd.merge(dfpqprop, tblxprop, how='left',
                     left_on=['año_fc', 'mes_fc', 'Propietario', 'Linea'], 
                     right_on=['año_fc', 'mes_fc', 'Propietario', 'Linea_pl'], sort=True)

In [89]:
##Para distinguir el uso de plataforma
dfpxpl = pd.read_excel ('Px pl 290920.xlsx')

dfpxpl = pd.melt(dfpxpl, id_vars=['Propietario',
                                 'Rango',
                                  'Año'],
        var_name='Mes',
        value_name='Precio_pl')

In [90]:
dfpxpl['Propietario'] = dfpxpl['Propietario'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [91]:
dfpxpl.dropna(subset=['Precio_pl'], inplace=True)

In [92]:
dfpxpl.head()

,Propietario,Rango,Año,Mes,Precio_pl
100,star-division,1000,2020,5,12.5
101,star-division,2000,2020,5,11.6
102,star-division,3000,2020,5,10.6
103,star-division,5000,2020,5,9.6
104,star-division,5001,2020,5,9.0


In [93]:
##Genero el rango de paquetes para uso de plataforma en el df paquetes
dfpqprop['Rango_pq'] = dfpqprop['Pq x prop']/1000 ##Divido por 1000 para redondear la unidad
dfpqprop['Rango_pq'] = dfpqprop['Rango_pq'].apply(np.ceil)*1000 ##Round up (ceil)
dfpqprop.loc[(dfpqprop.Rango_pq > 3000) & (dfpqprop.Rango_pq < 5000), 'Rango_pq'] = 5000
dfpqprop.loc[dfpqprop.Rango_pq > 5000, 'Rango_pq'] = 5001

In [94]:
dfpqprop['Código Externo'].loc[dfpqprop.Linea == 'Transacción'].groupby([dfpqprop.Linea, 
                                                                         dfpqprop.Propietario, 
                                                                         dfpqprop.año_fc, 
                                                                         dfpqprop.mes_fc , 
                                                                         dfpqprop.Rango_pq]).agg('count')

Linea        Propietario               año_fc  mes_fc  Rango_pq
Transacción  andesmar                  2020    4       1000.0        416
                                               5       2000.0       1355
                                               6       5000.0       3458
                                               7       5001.0       7202
                                               8       5001.0       8918
                                               9       5001.0       9131
                                               10      5001.0      15179
             dac_servicios_logisticos  2020    9       1000.0         10
                                               10      1000.0         29
             encopress                 2020    5       1000.0        148
                                               6       1000.0        634
                                               7       1000.0          6
             star-division             2020    5       1000.

In [95]:
dfpxpl['Mes'] = dfpxpl['Mes'].astype(float)
dfpxpl['Año'] = dfpxpl['Año'].astype(float)

In [96]:
##Cruzo el dataset de precios de plataforma según rango de Q paquetes
dfpqpxpl = pd.merge(dfpqprop, dfpxpl, how='left',
                     left_on=['año_fc', 'mes_fc', 'Propietario', 'Rango_pq'], 
                     right_on=['Año', 'Mes', 'Propietario', 'Rango'], sort=True)

In [97]:
dfpqpxpl.drop(['Año'], axis=1, inplace=True)
dfpqpxpl.drop(['Mes'], axis=1, inplace=True)

In [98]:
##Check group by
dfpqpxpl['Código Externo'].loc[dfpqpxpl.Empresa == 'unilever_mutual'].groupby(dfpqpxpl['Cant. de Bultos']).agg('count')

Cant. de Bultos
1.0     425
2.0     600
3.0     362
4.0      91
5.0      31
6.0       9
7.0       4
8.0       3
9.0       1
10.0      2
11.0      1
15.0      1
19.0      1
Name: Código Externo, dtype: int64

In [99]:
##
dfpqpxpl['Cant. de Bultos'].fillna(0, inplace=True)
dfpqpxpl['Cant. de Bultos'] = dfpqpxpl['Cant. de Bultos'].astype(int)

In [100]:
##Corrijo los tamaños de los paquetes según arreglo por cantidad de bultos
dfpqpxpl.loc[((dfpqpxpl.Empresa == 'unilever_mutual') & 
             ((dfpqpxpl['Cant. de Bultos'] == 1) | (dfpqpxpl['Cant. de Bultos'] == 2))), 'Tamaño'] = 'Chico'

dfpqpxpl.loc[((dfpqpxpl.Empresa == 'unilever_mutual') & 
              (dfpqpxpl['Cant. de Bultos'] > 2)), 'Tamaño'] = 'Mediano'

#Precio adicional (creo campo) por excedente de paquetes por sobre 6
dfpqpxpl.loc[((dfpqpxpl.Empresa == 'unilever_mutual') & 
              (dfpqpxpl['Cant. de Bultos'] > 6)), 'precio_adicional'] = 71 * (dfpqpxpl['Cant. de Bultos'] - 6)

#Default para historial sin Cantidad de bultos
dfpqpxpl.loc[((dfpqpxpl.Empresa == 'unilever_mutual') & (dfpqpxpl['Cant. de Bultos'] == 0)), 'Tamaño'] = 'Mediano'

In [101]:
##Corrijo los tamaños de los paquetes según arreglo por cantidad de bultos
dfpqpxpl.loc[((dfpqpxpl.Empresa == 'club_de_beneficios') & 
             ((dfpqpxpl['Cant. de Bultos'] == 1) | (dfpqpxpl['Cant. de Bultos'] == 2))), 'Tamaño'] = 'Chico'

dfpqpxpl.loc[((dfpqpxpl.Empresa == 'club_de_beneficios') & 
              (dfpqpxpl['Cant. de Bultos'] > 2)), 'Tamaño'] = 'Mediano'

#Precio adicional (creo campo) por excedente de paquetes por sobre 6
dfpqpxpl.loc[((dfpqpxpl.Empresa == 'club_de_beneficios') & 
              (dfpqpxpl['Cant. de Bultos'] > 6)), 'precio_adicional'] = 71 * (dfpqpxpl['Cant. de Bultos'] - 6)

dfpqpxpl['precio_adicional'].fillna(0, inplace=True)

#Default para historial sin Cantidad de bultos
dfpqpxpl.loc[((dfpqpxpl.Empresa == 'club_de_beneficios') & (dfpqpxpl['Cant. de Bultos'] == 0)), 'Tamaño'] = 'Mediano'

In [102]:
#Hay paquetes cargados como Store y son Enviar
dfpqpxpl.loc[((dfpqpxpl.Empresa == 'club_de_beneficios') & (dfpqpxpl['Tipo'] == 'store_withdrawal')), 'Tipo'] = 'enviar'

In [103]:
##Check group by
dfpqpxpl['Código Externo'].loc[dfpqpxpl.Empresa == 'unilever_mutual'].groupby(dfpqpxpl['Cant. de Bultos']).agg('count')

Cant. de Bultos
0      29
1     425
2     600
3     362
4      91
5      31
6       9
7       4
8       3
9       1
10      2
11      1
15      1
19      1
Name: Código Externo, dtype: int64

In [104]:
print(dfpqpxpl['Código Externo'].count() - len(dfpqpxpl['Código Externo'].unique()))
print(dfpqpxpl.loc[dfpqpxpl['Código Externo'] == 'FF185248832048'].shape[0])

7
1


In [105]:
dfpx.drop_duplicates(inplace=True)

In [106]:
##Para check cambio de precio
dfpqpxpl.loc[((dfpqpxpl['Cliente_facturacion']=='enviopack') & 
             (dfpqpxpl['mes_fc']==10))].shape[0] ###############

6488

In [107]:
dfpqpxpl.loc[((dfpqpxpl['Cliente_facturacion']=='enviopack') & 
             (dfpqpxpl['año_fc']==2020) &
             (dfpqpxpl['mes_fc']==10) & 
             (dfpqpxpl['dia_fc']<13)), 'mes_fc'] = 9

In [108]:
dfpqpxpl.loc[((dfpqpxpl['Cliente_facturacion']=='enviopack') & 
             (dfpqpxpl['mes_fc']==10))].shape[0] ###############

3855

In [109]:
dfpqpxpl[['año_fc', 'mes_fc', 'Tamaño', 'Plazo', 'ambito_geo_facturacion',
                              'Cliente_facturacion', 'Tipo', 'Linea']].duplicated().value_counts()

True     322255
False      3660
dtype: int64

In [110]:
dfpqpxpl['Tipo'].unique()

array(['enviar', 'store_withdrawal', 'flex'], dtype=object)

In [111]:
dfpx['Tipo_app_px'].unique()

array(['flex', 'enviar', 'store_withdrawal', nan], dtype=object)

In [112]:
##Cruzo con dataset precios paquetes
dfpqpx = pd.merge(dfpqpxpl, dfpx, how='left',
                     left_on=['año_fc', 'mes_fc', 'Tamaño', 'Plazo', 'ambito_geo_facturacion',
                              'Cliente_facturacion', 'Tipo', 'Linea'], 
                     right_on=['Año', 'Mes', 'Tamaño', 'Plazo', 'Ámbito Geográfico',
                               'Cliente', 'Tipo_app_px', 'Linea_px'], sort=True)

In [113]:
#Revierto el mes en los días de cambio de precio
#No es necesario indicar cliente, año, etc ... podría sólo corregir el mes de todos !!!!
dfpqpx.loc[((dfpqpx['Cliente_facturacion']=='enviopack') &
            (dfpqpxpl['año_fc']==2020) &
            (dfpqpx['mes_fc']==9) &
            (pd.DatetimeIndex(dfpqpx['fecha_fc']).month==10)), 'mes_fc'] = 10

In [114]:
#Check reversion
dfpqpx.loc[((dfpqpx['Cliente_facturacion']=='enviopack') & 
             (dfpqpx['mes_fc']==10))].shape[0] ###############

6488

In [115]:
print(dfpqpx['Código Externo'].count() - len(dfpqpx['Código Externo'].unique()))
print(dfpqpx.loc[dfpqpx['Código Externo'] == 'FF185248832048'].shape[0])

1876
1


In [116]:
dfcl = pd.read_excel ('Costo.xlsx')

In [117]:
dfcl.head()

,Plazo_pago,Tamaño_pago,OPL_pago,Zona_costo,Costo
0,Dia siguiente,Chico,PaQuery,Zona I,72.0
1,Dia siguiente,Grande,PaQuery,Zona I,122.2
2,Dia siguiente,Mediano,PaQuery,Zona I,94.0
3,Mismo dia,Chico,PaQuery,Zona I,120.0
4,Mismo dia,Grande,PaQuery,Zona I,202.8


In [118]:
dfcl['OPL_pago'] = dfcl['OPL_pago'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [119]:
cp = pd.read_csv('AR.csv', sep=";", encoding="latin1")

In [120]:
##Los paquers propios y de DAC no tienen cambio de zona, con algunas excepciones
dfpqpx.loc[((~dfpqpx['Nombre Paquer'].isin(['Cristian Gaston Orieta',
                                           'Enzo Polles',
                                           'Fernando Antonio Marin Boccone',
                                           'GL56 - Marcos Torres',
                                           'GL56 Miriam Veronica Frappa',
                                           'Javier Luis Cabrera',
                                           'JONATAN FERNANDO ORIETA','Luis Eleuterio Orieta'])) 
            & (dfpqpx['OPL Paquer'] == 'paquery') | (dfpqpx['OPL Paquer'] == 'dac_servicios_logisticos')), 
           'Zona_clasif_OPL'] = 'Zona I'

###CAMBIO A MINUSCULAS, ETC

In [121]:
dfpqpx['OPL_pago'].unique()

array(['paquery', 'otro', 'star-division'], dtype=object)

In [122]:
dfpqproppxcl = pd.merge(dfpqpx, dfcl, how='left',
                       left_on=['Zona_clasif_OPL', 'Tamaño', 'Plazo', 'OPL_pago'],
                       right_on=['Zona_costo', 'Tamaño_pago', 'Plazo_pago', 'OPL_pago'], sort=True)

In [123]:
##Las transacciones no tienen costo para Paquery
dfpqproppxcl.loc[dfpqproppxcl.Linea == 'Transacción', 'Costo'] = 0

##Los paquery points que no opera PaQuery no tienen costo de paquers,
##le pagan los OPLs que lo operan (hay que corregir esto, hoy se identifican por zonas)
dfpqproppxcl.loc[((~dfpqproppxcl['ambito_geo_costos'].isin(['caba',
                                           'norte',
                                           'sur',
                                           'oeste',
                                           'la_plata',
                                           'campana',
                                           'lujan', 'mar_del_plata'])) 
            & (dfpqproppxcl['OPL Paquer'] == 'paquery') & (dfpqproppxcl['OPL Entrega'] == 'paquery')), 
           'Costo'] = 0

In [124]:
dfpqproppxcl['Precio'].fillna(0, inplace=True)
dfpqproppxcl['Precio_pl'].fillna(0, inplace=True)
dfpqproppxcl['Propietario'].fillna('paquery', inplace=True) ##El propietario de los clientes de envioPack xej son de PaQuery

In [125]:
print(dfpqproppxcl['Código Externo'].count() - len(dfpqproppxcl['Código Externo'].unique()))
print(dfpqproppxcl.loc[dfpqproppxcl['Código Externo'] == 'FF185248832048'].shape[0])

1876
1


In [126]:
dfst = pd.read_excel ('px_star.xlsx')

In [127]:
dfst.head()

,Propietario,zona_star,plazo_star,tamaño_star,precio_star
0,Star-division,Zona I,Dia siguiente,Chico,106
1,Star-division,Zona I,Dia siguiente,Grande,171
2,Star-division,Zona I,Dia siguiente,Mediano,134
3,Star-division,Zona I,Mismo dia,Chico,132
4,Star-division,Zona I,Mismo dia,Grande,215


In [128]:
dfst['Propietario'] = dfst['Propietario'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [129]:
##Paquetes + precio de paquetes para clientes de Star-division
dfult = pd.merge(dfpqproppxcl, dfst, how='left',
                       left_on=['Propietario','Zona_clasif_OPL', 'Plazo', 'Tamaño'],
                       right_on=['Propietario', 'zona_star', 'plazo_star', 'tamaño_star'], sort=True)

In [130]:
##Completo con 0 la columna precio_star para todos los propietarios que no son Star
dfult['precio_star'].fillna(0, inplace=True)

##Reemplazo por 0 los que correponden a transacciones de Star
dfult.loc[dfult['OPL Entrega'] == 'star-division', 'precio_star'] = 0

In [131]:
dfplus = pd.read_excel ('Plus paquers.xlsx')

In [132]:
dfplus.head()

,opl_entrega_plus,zona_plus,año_plus,mes_plus,plus
0,paquery,la_plata,2020,1,20
1,paquery,la_plata,2020,2,20
2,paquery,la_plata,2020,3,20
3,paquery,la_plata,2020,4,20
4,paquery,la_plata,2020,5,20


In [133]:
dfplus['opl_entrega_plus'] = dfplus['opl_entrega_plus'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfplus['zona_plus'] = dfplus['zona_plus'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [134]:
dfult.columns

Index(['1er Visita', '2da Visita', 'Arribo a Paquery Point', 'Cant. de Bultos',
       'Codigo Postal', 'Costo Estimado', 'Creación', 'Código Externo',
       'Descripción', 'Destino Comentarios', 'Destino Dir Comentarios',
       'Dirección Origen', 'Dirección destino', 'Empresa', 'Estado',
       'Fecha de Ingreso a Paquery', 'Fecha de entrega', 'Fecha en Poder',
       'Motivo 1er visita', 'Motivo 2da visita', 'Nombre Origen',
       'Nombre Paquer', 'Nombre destinatario', 'OPL Entrega', 'OPL Paquer',
       'Paquer', 'Paquery Point', 'Plazo', 'Rollcontainer/Posición', 'Tamaño',
       'Tipo', 'Usuario', 'Zona', 'Clientes 3ero', 'Cliente_mayorista',
       'Cliente_facturacion', 'Zona_ppal', 'Zona_sec', 'Zona_cod_def',
       'cod_default', 'cod_postal', 'Zona_cod', 'ambito_geo_facturacion',
       'ambito_geo_costos', 'Paquery_Point', 'Zona_pqp', 'OPL_default_pqp',
       'Zona_clasif_OPL', 'Zona_cod_def_ambito', 'ambito_geo_fact_def',
       'Cliente_propietario', 'Propietario', '

In [135]:
##Plus de pago a paquers por zona 
dfult = pd.merge(dfult, dfplus, how='left',
                       left_on=['OPL Entrega','Zona', 'año_fc', 'mes_fc'],
                       right_on=['opl_entrega_plus', 'zona_plus', 'año_plus', 'mes_plus'], sort=True)

##Fill con 0 e incorporo al costo
dfult['plus'].fillna(0, inplace=True)
dfult['Costo'] = dfult['Costo'] + dfult['plus']

In [136]:
##Excepciones
dfult['Costo Estimado'].fillna(0, inplace=True)
dfult['Costo Estimado'] = dfult['Costo Estimado'].astype(int)

dfult.loc[((dfult['Nombre Paquer'] == 'Gustavo Santillan') &
           (dfult['Empresa'] == 'enviopack') &
           (dfult['Costo Estimado'] > 30000)), 
           'Costo'] = 200

In [137]:
dfreg = pd.read_excel ('region.xlsx')

In [138]:
dfreg.columns

Index(['zona_region', 'region'], dtype='object')

In [139]:
dfreg['zona_region'] = dfreg['zona_region'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [140]:
dfult = pd.merge(dfult, dfreg, how='left',
                       left_on=['Zona'],
                       right_on=['zona_region'], sort=True)

dfult['region'].fillna('interior', inplace=True)

In [141]:
dfult.loc[(dfult['region'] == 'interior'), 'Costo'] = 0

In [142]:
dfult.shape

(327784, 90)

In [143]:
dfult.columns

Index(['1er Visita', '2da Visita', 'Arribo a Paquery Point', 'Cant. de Bultos',
       'Codigo Postal', 'Costo Estimado', 'Creación', 'Código Externo',
       'Descripción', 'Destino Comentarios', 'Destino Dir Comentarios',
       'Dirección Origen', 'Dirección destino', 'Empresa', 'Estado',
       'Fecha de Ingreso a Paquery', 'Fecha de entrega', 'Fecha en Poder',
       'Motivo 1er visita', 'Motivo 2da visita', 'Nombre Origen',
       'Nombre Paquer', 'Nombre destinatario', 'OPL Entrega', 'OPL Paquer',
       'Paquer', 'Paquery Point', 'Plazo', 'Rollcontainer/Posición', 'Tamaño',
       'Tipo', 'Usuario', 'Zona', 'Clientes 3ero', 'Cliente_mayorista',
       'Cliente_facturacion', 'Zona_ppal', 'Zona_sec', 'Zona_cod_def',
       'cod_default', 'cod_postal', 'Zona_cod', 'ambito_geo_facturacion',
       'ambito_geo_costos', 'Paquery_Point', 'Zona_pqp', 'OPL_default_pqp',
       'Zona_clasif_OPL', 'Zona_cod_def_ambito', 'ambito_geo_fact_def',
       'Cliente_propietario', 'Propietario', '

In [144]:
dfc_and.columns

Index(['Codigo Postal', 'Provincia', 'TRONCAL', 'ULTIMA MILLA', 'Operador',
       'Tamaño_and', 'Tamaño_and_2', 'peso_and', 'zona_and', 'año_and',
       'mes_and', 'costo_tron', 'sub-zona_and', 'costo_ent'],
      dtype='object')

In [145]:
dfc_and.columns

Index(['Codigo Postal', 'Provincia', 'TRONCAL', 'ULTIMA MILLA', 'Operador',
       'Tamaño_and', 'Tamaño_and_2', 'peso_and', 'zona_and', 'año_and',
       'mes_and', 'costo_tron', 'sub-zona_and', 'costo_ent'],
      dtype='object')

In [146]:
dfc_and.head()

,Codigo Postal,Provincia,TRONCAL,ULTIMA MILLA,Operador,Tamaño_and,Tamaño_and_2,peso_and,zona_and,año_and,mes_and,costo_tron,sub-zona_and,costo_ent
0,2000,SANTA FE,Zona I,Cabecera,LH-Paquery,Chico,Hasta 80cm lineales,Hasta 5kg,Zona I,2020,1,96.0,Cabecera,107.0
1,2121,SANTA FE,Zona I,Cabecera,Andesmar,Chico,Hasta 80cm lineales,Hasta 5kg,Zona I,2020,1,96.0,Cabecera,107.0
2,3000,SANTA FE,Zona I,Cabecera,Andesmar,Chico,Hasta 80cm lineales,Hasta 5kg,Zona I,2020,1,96.0,Cabecera,107.0
3,3100,ENTRE RIOS,Zona I,Cabecera,Andesmar,Chico,Hasta 80cm lineales,Hasta 5kg,Zona I,2020,1,96.0,Cabecera,107.0
4,7600,BUENOS AIRES,Zona I,Cabecera,Andesmar,Chico,Hasta 80cm lineales,Hasta 5kg,Zona I,2020,1,96.0,Cabecera,107.0


In [147]:
dfult['Codigo Postal'] = dfult['Codigo Postal'].astype(int)

In [148]:
dfult.dtypes

1er Visita                datetime64[ns]
2da Visita                datetime64[ns]
Arribo a Paquery Point    datetime64[ns]
Cant. de Bultos                    int32
Codigo Postal                      int32
                               ...      
año_plus                         float64
mes_plus                         float64
plus                             float64
zona_region                       object
region                            object
Length: 90, dtype: object

In [149]:
dfult['año_fc'] = dfult['año_fc'].astype(int)
dfult['mes_fc'] = dfult['mes_fc'].astype(int)

dfc_and['año_and'] = dfc_and['año_and'].astype(int)
dfc_and['mes_and'] = dfc_and['mes_and'].astype(int)

In [150]:
dfult2 = pd.merge(dfult, dfc_and, how='left',
                       left_on=['año_fc', 'mes_fc', 'Codigo Postal', 'Tamaño'],
                       right_on=['año_and', 'mes_and', 'Codigo Postal', 'Tamaño_and'], sort=True)

In [151]:
dfult.shape

(327784, 90)

In [152]:
dfult2.shape

(327784, 103)

In [153]:
dfc_and['Tamaño_and'].value_counts()

Grande      4030
Grande 2    4030
Mediano     4030
Mini        4030
Chico       4030
Name: Tamaño_and, dtype: int64

In [154]:
dfult['Tamaño'].value_counts()

Chico      225921
Mediano     68783
Grande      33080
Name: Tamaño, dtype: int64

In [155]:
dfult2['Costo'].loc[(dfult2['region'] == 'interior')].value_counts()

0.0    27244
Name: Costo, dtype: int64

In [156]:
dfult2['Costo'] = np.where(dfult2['region'] == 'interior', dfult2['costo_ent'] + dfult2['costo_tron'], dfult2['Costo'])

In [157]:
dfult2[['Tamaño', 'Codigo Postal', 'Costo', 'Mes', 'Año']].loc[(dfult2['region'] == 'interior') & 
                                                               (dfult2['Costo'].isnull())].groupby('Codigo Postal').agg('count')

,Tamaño,Costo,Mes,Año
Codigo Postal,,,,
2700,1,0,0,0
2901,1,0,0,0
3006,1,0,0,0
3300,2,0,1,1
4401,1,0,0,0


In [158]:
dfult2['Costo'].isnull().groupby(dfult2['Codigo Postal']).agg('count')

Codigo Postal
1000     237
1001     236
1002    2955
1003      92
1004     152
        ... 
7603     251
7605      41
8000      21
8300      11
8500       3
Name: Costo, Length: 670, dtype: int64

In [159]:
dfult2['Costo'].loc[(dfult2['region'] == 'interior')].isnull().value_counts()

False    27238
True         6
Name: Costo, dtype: int64

In [160]:
dfult2.loc[(dfult2['Linea'] == 'Transacción'), 'Costo'] = 0

In [161]:
#Precio adicional de Club de beneficios y Unilever
dfult2['Precio'] = dfult2['Precio'] + dfult2['precio_adicional']

In [162]:
dfult2['Costo'].loc[(dfult2['Costo']==0) & (dfult2['mes_fc']==9)].value_counts()

0.0    11569
Name: Costo, dtype: int64

In [163]:
dfcom = pd.read_excel ('comisiones.xlsx')

In [164]:
dfcom['empresa'] = dfcom['empresa'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
dfcom['Tipo_comision'] = dfcom['Tipo_comision'].str.lower().str.strip().str.replace(' ', '_').str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [165]:
dfult2 = pd.merge(dfult2, dfcom, how='left',
                       left_on=['Cliente_facturacion', 'Tipo'],
                       right_on=['empresa', 'Tipo_comision'], sort=True)

In [166]:
inicio = datetime.now()
dfult2.to_excel("Paq x paq wip7.xlsx")
datetime.now() - inicio

datetime.timedelta(seconds=1228, microseconds=345951)